In [1]:
import json
import torch
import torch.nn.functional as F
import numpy as np

c:\Users\Admin\anaconda3\envs\vir_py3.7\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
with open("./data/Wemap_Category_url.json", encoding="utf-8", mode="r") as f:
    categorys = list(json.load(f).keys())

In [2]:
with open("./data/eng/스콥카테고리.json", encoding="utf-8", mode="r") as f:
    json_file = json.load(f)
    big_category = json_file.keys()
    categorys = list(big_category) \
                + [cur for key in big_category for cur in json_file[key]]\
                + [key + " " + cur for key in big_category for cur in json_file[key]]
    

In [3]:
from konlpy.tag import Okt
okt = Okt()

In [30]:
# 위메프 regex
import re
def apply_regex_wemap(text):
    category_list = text.split("<")
    
    out_category = []
    for cur in category_list:
        out_category.append(re.sub('[<>/]', " ", cur))
    okt_noun_list = okt.nouns(" ".join(out_category[1:]))
    return " ".join(okt_noun_list)

In [4]:
# 스콥 regex
import re
def apply_regex_scop(text):
    # category_list = text.split(" ")

    # out_category = []
    # for cur in category_list:
    #     out_category.append(re.sub('[<>/]', " ", cur))

    # okt_noun_list = okt.nouns(" ".join(out_category))
    return text #" ".join(okt_noun_list)

In [5]:
# total_category = [apply_regex_wemap(cur) for cur in categorys]
total_category = [apply_regex_scop(cur) for cur in categorys]

In [6]:
with open("./data/eng/nodes.txt", encoding="utf-8", mode="r") as f:
    tags = [re.sub("\n","",cur) for cur in f.readlines()]

In [7]:
from transformers import BertTokenizer, BertModel

# 한글
# tokenizer = BertTokenizer.from_pretrained("snunlp/KR-BERT-char16424")
# model = BertModel.from_pretrained("snunlp/KR-BERT-char16424")

# 영어
tokenizer = BertTokenizer.from_pretrained("snunlp/KR-BERT-char16424")
model = BertModel.from_pretrained("snunlp/KR-BERT-char16424")

Some weights of the model checkpoint at snunlp/KR-BERT-char16424 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# TFIDF 테이블 만들기

In [8]:
def get_token(sentence):
    tokens = tokenizer.encode(sentence, add_special_tokens=True, return_tensors="pt").tolist()[0]
    return " ".join([str(token) for token in tokens])

In [9]:
token_list = [get_token(category) for category in total_category+tags]

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(token_pattern=r'\b\w+\b')

tfidf_vectors = vectorizer.fit_transform(token_list)
tfidf_words = vectorizer.get_feature_names_out()

# TFIDF 기반 임베딩

In [11]:
# 프로그레스 바를 출력하는 함수
def print_progress_bar(iteration, total, prefix='Complete', suffix='', decimals=5, length=100, fill='█', print_end='\r'):
    """
    Parameters:
        iteration (int): 현재 진행된 작업 수
        total (int): 전체 작업 수
        prefix (str): 프로그레스 바 앞에 추가할 텍스트
        suffix (str): 프로그레스 바 뒤에 추가할 텍스트
        decimals (int): 백분율에서 소수점 이하 자릿수
        length (int): 프로그레스 바의 총 길이 (글자 수)
        fill (str): 프로그레스 바의 채우는 문자
        print_end (str): 프로그레스 바 출력 후 커서의 위치를 조정하는 문자열
    """
    # 현재 진행된 작업의 백분율 계산
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))

    # 프로그레스 바의 채워진 길이 계산
    filled_length = int(length * iteration // total)

    # 프로그레스 바 출력
    bar = fill * filled_length + '-' * (length - filled_length)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end=print_end)

    # 모든 작업이 끝나면 줄바꿈
    if iteration == total:
        print()

- TFIDF O

In [12]:
def get_targettokens_and_TFIDFvalues(tokens, st_idx):
    token_idx = tfidf_vectors[st_idx].indices
    token_id_pairdict = dict(zip(tfidf_words[token_idx], token_idx))

    target_tokens = []
    for token in tokens[0].tolist():
        if str(token) in list(token_id_pairdict.keys()):
            target_tokens.append(token)
    
    tfidf_value_list =[tfidf_vectors[st_idx, token_id_pairdict[str(token)]] for token in target_tokens]

    return torch.IntTensor(target_tokens), torch.FloatTensor(tfidf_value_list)

In [13]:
def get_sentence_vector_tfidf(sentence, st_idx):
    tokens = tokenizer.encode(sentence, add_special_tokens=True, return_tensors="pt")
    target_tokens, tfidf_value_list = get_targettokens_and_TFIDFvalues(tokens, st_idx)

    with torch.no_grad():
        model_output = model(target_tokens.reshape(1,-1))
        last_hidden_states = model_output[0]

    st_tfidf_vec = last_hidden_states * tfidf_value_list.reshape(-1,1)
    sentence_vector = torch.mean(st_tfidf_vec, dim=1)

    return sentence_vector[0]

In [14]:
def get_sentence_matrix_tfidf(sentence_list, st_idx_plus = 0):
    
    total = len(sentence_list)
    iteration = 0
    
    sentence_vector_list = []
    
    for idx, sentence in enumerate(sentence_list):
        try:
            sentence_vector_list.append(get_sentence_vector_tfidf(sentence, idx + st_idx_plus))

            iteration += 1
            print_progress_bar(iteration, total)
        except:
            sentence_vector_list.append(torch.zeros(len(sentence_vector_list[0])))
            print("error")
    
    return sentence_vector_list

- TFIDF X

In [42]:
def get_sentence_vector(sentence):
    tokens = tokenizer.encode(sentence, add_special_tokens=True, return_tensors="pt")

    with torch.no_grad():
        model_output = model(tokens.reshape(1,-1))
        last_hidden_states = model_output[0]
    sentence_vector = torch.mean(last_hidden_states, dim=1)
    return sentence_vector[0]

In [43]:
def get_sentence_matrix(sentence_list):
    
    total = len(sentence_list)
    iteration = 0
    
    sentence_vector_list = []
    for idx, sentence in enumerate(sentence_list):

        sentence_vector_list.append(get_sentence_vector(sentence))

        iteration += 1
        print_progress_bar(iteration, total)
    
    return sentence_vector_list

# 문장벡터 생성

In [15]:
category_st_vec_list = get_sentence_matrix_tfidf(total_category)
# category_st_vec_list = get_sentence_matrix(total_category)

Complete |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% 


In [19]:
tag_stvec_list = get_sentence_matrix_tfidf(tags, len(categorys))
# tag_stvec_list = get_sentence_matrix(tags)

In [46]:
def cal_cosine_sim(vector1, vector2):
    similarity = F.cosine_similarity(vector1, torch.stack(vector2, dim=0))
    return similarity

In [47]:
category_and_tag_similarity = []

total = len(category_st_vec_list)
iteration = 0

for category_vec in category_st_vec_list:
    cos_sim = cal_cosine_sim(category_vec, tag_stvec_list)
    category_and_tag_similarity.append(cos_sim.tolist())

    iteration += 1
    print_progress_bar(iteration, total)

Complete |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00000% 


In [48]:
category_tag_sim_dict = {}

for idx, category in enumerate(total_category):

    tag_sim_pair = list(zip(tags, category_and_tag_similarity[idx]))
    sorted_tag_sim_pair = sorted(tag_sim_pair, key = lambda x:x[1], reverse= True)
    category_tag_sim_dict[category] = sorted_tag_sim_pair

In [49]:
with open("./data/category_tag_sim(TFIDF).json", encoding="utf-8", mode="w") as f:
    json.dump(category_tag_sim_dict, f, ensure_ascii=False, indent=4)

In [50]:
category_tag_sim_dict.keys()

dict_keys(['브랜드 여성 의류 원피스', '브랜드 여성 의류 티셔츠', '브랜드 여성 의류 블라우스 셔츠', '브랜드 여성 의류 투맨 후드', '브랜드 여성 의류 니트 가디건 조끼', '브랜드 여성 의류 자켓 코트', '브랜드 여성 의류 패딩 점퍼 상', '브랜드 여성 의류 가죽 모피 무스 탕', '브랜드 여성 의류 정장 세트', '브랜드 여성 의류 스커트 치마', '브랜드 여성 의류 캐 주얼 바지 팬츠', '브랜드 여성 의류 청바지', '브랜드 여성 의류 트레이닝복', '브랜드 여성 의류 수영복 비치 웨어', '브랜드 남성 의류 티셔츠', '브랜드 남성 의류 셔츠 남방', '브랜드 남성 의류 투맨 후드', '브랜드 남성 의류 니트 가디건 조끼', '브랜드 남성 의류 자켓 코트', '브랜드 남성 의류 패딩 점퍼 상', '브랜드 남성 의류 정장', '브랜드 남성 의류 캐 주얼 바지 팬츠', '브랜드 남성 의류 청바지', '브랜드 남성 의류 수영복 비치 웨어', '브랜드 진 캐 주얼 티셔츠 셔츠', '브랜드 진 캐 주얼 원피스 스커트', '브랜드 진 캐 주얼 투맨 후드', '브랜드 진 캐 주얼 니트 가디건 조끼', '브랜드 진 캐 주얼 자켓 코트', '브랜드 진 캐 주얼 패딩 점퍼 상', '브랜드 진 캐 주얼 팬츠', '브랜드 언더웨어 잠옷 홈웨어 웨어', '브랜드 언더웨어 남성 팬티', '브랜드 언더웨어 남성 내의', '브랜드 언더웨어 브라 탑 런닝 바지', '브랜드 언더웨어 여성 팬티', '브랜드 언더웨어 여성 내의', '브랜드 언더웨어 남성 런닝', '브랜드 언더웨어 보정속옷', '브랜드 언더웨어 여성 브라', '브랜드 언더웨어 여성 브라 팬티 세트', '브랜드 언더웨어 테마 속옷', '브랜드 언더웨어 속옷 악세사리', '브랜드 가방 잡화 가방', '브랜드 가방 잡화 지갑 벨트', '브랜드 가방 잡화 패션 잡화 소품', '브랜드 가방 잡화 선글라스 안경 테', '브랜드 신발 여성화', '브랜드 신발 남성화', '브랜드 신발 캐 주얼', '브랜드 신발